Папка на GDrive:

https://drive.google.com/drive/folders/1FeUwrVfxp09xAifiMngwtfrRZ8nhtj8T?usp=sharing


In [1]:
from os import listdir

USE_GDRIVE = False


if USE_GDRIVE:
    from google.colab import drive
    drive.mount("/home/GDrive")
    DATA_PATH = f"/home/GDrive/MyDrive/made_2021_fraud_project/data"
else:
    DATA_PATH = "../../data"

listdir(DATA_PATH)

['unlabeled',
 'good_data',
 'all_gps.csv',
 'all_accel.csv',
 'drivers_with_gps_and_sl_problems',
 'rides_of_drivers.csv']

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import folium
from haversine import haversine
from os import listdir


In [3]:
df = pd.read_csv(f"{DATA_PATH}/rides_of_drivers.csv", index_col=[0], parse_dates=[1, 2, 3, 4, 5])

Немного о статусах:
- ORDER_DTTM - время когда клиент нажал заказать такси
- DRIVER_ASSIGNED_DTTM - водитель назначен
- READY_FOR_COLL_DTTM - водитель приехал и готов забрать клиента
- CLIENT_COLL_DTTM - водитель забрал клиента
- ORDER_FINISHED_DTTM - водитель нажал на клавишу завершить заказ
- A_LAT, A_LONG - координаты точки А
- B_LAT, B_LONG - координаты точки B

In [4]:
df.describe(datetime_is_numeric=True).T

,count,mean,min,25%,50%,75%,max,std
ORDER_DTTM,8036,2021-04-07 04:12:24.445619712,2020-10-25 06:58:39,2021-01-13 09:25:10.250000128,2021-04-11 19:05:45.500000,2021-06-18 04:59:32.750000128,2021-08-29 23:20:16,NaN
DRIVER_ASSIGNED_DTTM,8036,2021-04-07 04:20:39.646217216,2020-10-25 06:59:00,2021-01-13 09:26:58.249999872,2021-04-11 19:06:13,2021-06-18 06:37:03.500000256,2021-08-29 23:20:52,NaN
READY_FOR_COLL_DTTM,7112,2021-04-07 07:13:17.572694016,2020-10-25 07:02:35,2021-01-13 13:36:07.249999872,2021-04-11 20:25:03.500000,2021-06-18 06:42:25.750000128,2021-08-29 23:24:59,NaN
CLIENT_COLL_DTTM,6930,2021-04-06 21:23:01.645454592,2020-10-25 07:04:52,2021-01-13 10:47:16.249999872,2021-04-11 01:31:21.500000,2021-06-17 12:49:23.500000256,2021-08-29 23:27:21,NaN
ORDER_FINISHED_DTTM,6926,2021-04-06 21:28:29.464048896,2020-10-25 07:20:26,2021-01-13 10:45:27.750000128,2021-04-11 09:22:31,2021-06-17 13:35:00.750000128,2021-08-29 23:36:12,NaN
A_LAT,8036,55.7566,55.3593,55.6767,55.7598,55.8518,56.7507,0.122224
A_LON,8036,37.592,36.3458,37.4897,37.574,37.6798,38.4633,0.170441
B_LAT,8027,55.7554,55.1654,55.6747,55.758,55.8515,56.7502,0.12308
B_LON,8027,37.5916,36.1347,37.4868,37.5766,37.6842,38.891,0.177557
driver_hash,8036,1.842e+17,-9.21858e+18,-4.39057e+18,-1.60215e+17,5.06087e+18,9.20647e+18,5.40427e+18


In [5]:
df["DRIVER_ASSIGNED_TIME"] = df["DRIVER_ASSIGNED_DTTM"] - df["ORDER_DTTM"]
df["DRIVER_ARRIVED_TIME"] =  df["READY_FOR_COLL_DTTM"] - df["DRIVER_ASSIGNED_DTTM"]
df["CLINET_ARRIVED_TIME"] = df["CLIENT_COLL_DTTM"] - df["READY_FOR_COLL_DTTM"]
df["ORDER_DRIVING_TIME"] = df["ORDER_FINISHED_DTTM"] - df["CLIENT_COLL_DTTM"]
df.describe(datetime_is_numeric=True).T

,count,mean,min,25%,50%,75%,max,std
ORDER_DTTM,8036,2021-04-07 04:12:24.445619712,2020-10-25 06:58:39,2021-01-13 09:25:10.250000128,2021-04-11 19:05:45.500000,2021-06-18 04:59:32.750000128,2021-08-29 23:20:16,NaN
DRIVER_ASSIGNED_DTTM,8036,2021-04-07 04:20:39.646217216,2020-10-25 06:59:00,2021-01-13 09:26:58.249999872,2021-04-11 19:06:13,2021-06-18 06:37:03.500000256,2021-08-29 23:20:52,NaN
READY_FOR_COLL_DTTM,7112,2021-04-07 07:13:17.572694016,2020-10-25 07:02:35,2021-01-13 13:36:07.249999872,2021-04-11 20:25:03.500000,2021-06-18 06:42:25.750000128,2021-08-29 23:24:59,NaN
CLIENT_COLL_DTTM,6930,2021-04-06 21:23:01.645454592,2020-10-25 07:04:52,2021-01-13 10:47:16.249999872,2021-04-11 01:31:21.500000,2021-06-17 12:49:23.500000256,2021-08-29 23:27:21,NaN
ORDER_FINISHED_DTTM,6926,2021-04-06 21:28:29.464048896,2020-10-25 07:20:26,2021-01-13 10:45:27.750000128,2021-04-11 09:22:31,2021-06-17 13:35:00.750000128,2021-08-29 23:36:12,NaN
A_LAT,8036,55.7566,55.3593,55.6767,55.7598,55.8518,56.7507,0.122224
A_LON,8036,37.592,36.3458,37.4897,37.574,37.6798,38.4633,0.170441
B_LAT,8027,55.7554,55.1654,55.6747,55.758,55.8515,56.7502,0.12308
B_LON,8027,37.5916,36.1347,37.4868,37.5766,37.6842,38.891,0.177557
driver_hash,8036,1.842e+17,-9.21858e+18,-4.39057e+18,-1.60215e+17,5.06087e+18,9.20647e+18,5.40427e+18


In [6]:
df_gps = pd.read_csv(f"{DATA_PATH}/all_gps.csv", parse_dates=[0, 3])
df_gps

,time,lat,lon,gps_time,driver_hash,fraud
0,2021-04-13 11:11:07,55.765980,37.707788,2021-04-13 11:11:07,2025312357143911724,1.0
1,2021-04-13 11:11:08,55.765980,37.707788,2021-04-13 11:11:07,2025312357143911724,1.0
2,2021-04-13 11:11:08,55.754542,37.697707,2021-04-13 11:11:07,2025312357143911724,1.0
3,2021-04-13 11:11:08,55.754542,37.697707,2021-04-13 11:11:08,2025312357143911724,1.0
4,2021-04-13 11:11:10,55.754542,37.697707,2021-04-13 11:11:09,2025312357143911724,1.0
...,...,...,...,...,...,...
5050287,2021-02-05 18:32:06,55.935577,37.517716,2021-02-05 18:32:06,-2035413153418187612,NaN
5050288,2021-02-05 18:32:06,55.935580,37.517710,2021-02-05 18:32:06,-2035413153418187612,NaN
5050289,2021-02-05 18:32:09,55.935572,37.517723,2021-02-05 18:32:09,-2035413153418187612,NaN
5050290,2021-02-05 18:32:10,55.935572,37.517724,2021-02-05 18:32:10,-2035413153418187612,NaN


In [9]:
counter = 0
driver_ids = df_gps["driver_hash"].unique()
for idx in driver_ids:
    if df[df["driver_hash"] == idx].shape[0] > 0:
        counter += 1
        
display(f"On gps merged {counter}/{len(driver_ids)} drivers")

'On gps merged 272/317 drivers'

In [8]:
df_accel = pd.read_csv(f"{DATA_PATH}/all_accel.csv", parse_dates=[0, 3])
df_accel

,time,x,y,z,lat,lon,driver_hash,fraud
0,2021-04-12 11:18:26.068266,0.931344,9.692925,2.15957,55.756474,37.695483,2025312357143911724,1.0
1,2021-04-12 11:18:26.070640,0.933739,9.665392,2.0769699,55.756474,37.695483,2025312357143911724,1.0
2,2021-04-12 11:18:26.072441,0.946907,9.671378,2.1080947,55.756454,37.695475,2025312357143911724,1.0
3,2021-04-12 11:18:26.073274,0.938527,9.676167,2.099715,55.756454,37.695470,2025312357143911724,1.0
4,2021-04-12 11:18:26.074903,0.942118,9.677363,2.0901381999999997,55.756454,37.695470,2025312357143911724,1.0
...,...,...,...,...,...,...,...,...
2172984,2021-02-06 18:07:59.180188,-1.386353,8.256638,4.687729,55.817333,37.566908,-2035413153418187612,NaN
2172985,2021-02-06 18:07:59.182144,-1.230728,8.393112,4.2543793,55.817330,37.566903,-2035413153418187612,NaN
2172986,2021-02-06 18:07:59.182711,-1.222351,8.415863,4.1562195,55.817332,37.566905,-2035413153418187612,NaN
2172987,2021-02-06 18:07:59.183246,-1.944199,8.464935,4.336975,55.817331,37.566905,-2035413153418187612,NaN


In [10]:
counter = 0
driver_ids = df_accel["driver_hash"].unique()
for idx in driver_ids:
    if df[df["driver_hash"] == idx].shape[0] > 0:
        counter += 1
        
display(f"On accel merged {counter}/{len(driver_ids)} drivers")

'On accel merged 269/314 drivers'